In [2]:
#Загрузка библиотек
import numpy as np
import pandas as pd
import torch
import re

In [5]:
#Загрузчик с диска гугл
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train = pd.read_csv('../content/drive/MyDrive/Colab Notebooks/Torch/6/train.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:
test = pd.read_csv('../content/drive/MyDrive/Colab Notebooks/Torch/6/test.csv')
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [8]:
train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [9]:
df_train = train
df_test = test

In [11]:
df_train['tweet'] = df_train['tweet'].apply (lambda x: " ".join (x.lower () for x in x.split ()))

In [13]:
df_train['tweet'] = df_train['tweet'].str.replace (r"""[^\w\s]+""","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words ('english')
df_train['tweet'] = df_train['tweet'].apply (lambda x: " ".join (x for x in x.split () if x not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
from textblob import TextBlob
nltk.download('punkt')
TextBlob (df_train['tweet'][3]).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


WordList(['model', 'love', 'u', 'take', 'u', 'time', 'urð', 'ðððð', 'ððð'])

In [20]:
from nltk.stem import PorterStemmer
st = PorterStemmer ()
df_train['tweet'][:5].apply (lambda x: " ".join ([st.stem(word) for word in x.split ()]))

0    user father dysfunct selfish drag kid dysfunct...
1    user user thank lyft credit cant use caus dont...
2                                       bihday majesti
3                model love u take u time urð ðððð ððð
4                              factsguid societi motiv
Name: tweet, dtype: object

In [24]:
from textblob import Word
nltk.download('wordnet')
nltk.download('omw-1.4')
df_train['tweet'] = df_train['tweet'].apply (lambda x: " ".join ([Word(word).
                                                     lemmatize () for word in x.split ()]))

df_train['tweet']

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


0        user father dysfunctional selfish drag kid dys...
1        user user thanks lyft credit cant use cause do...
2                                           bihday majesty
3                    model love u take u time urð ðððð ððð
4                            factsguide society motivation
                               ...                        
31957                        ate user isz youuuðððððððððâï
31958    see nina turner airwave trying wrap mantle gen...
31959       listening sad song monday morning otw work sad
31960    user sikh temple vandalised calgary wso condem...
31961                                    thank user follow
Name: tweet, Length: 31962, dtype: object

In [31]:
max_words = 2000
max_len = 20
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [33]:
tokens_filtered = [word for word in df_train['tweet'] if word.isalnum()]

In [34]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [35]:
tokens_filtered_top[:10]

['user',
 'make',
 'bihday',
 'sad',
 'love',
 'smile',
 'friday',
 'place',
 'wish',
 'worrying']

In [36]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [41]:
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [39]:
df_test['tweet'] = df_test['tweet'].apply (lambda x: " ".join (x.lower () for x in x.split ()))
df_test['tweet'] = df_test['tweet'].str.replace (r"""[^\w\s]+""","")
df_test['tweet'] = df_test['tweet'].apply (lambda x: " ".join (x for x in x.split () if x not in stop))
TextBlob (df_test['tweet'][3]).words
df_test['tweet'][:5].apply (lambda x: " ".join ([st.stem(word) for word in x.split ()]))
df_test['tweet'] = df_test['tweet'].apply (lambda x: " ".join ([Word(word).
                                                     lemmatize () for word in x.split ()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [43]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]], dtype=np.int32)

CPU times: user 6.7 s, sys: 12.7 ms, total: 6.71 s
Wall time: 7.57 s


In [44]:
x_train.shape

(31962, 20)

In [49]:
x_train[0]

array([ 1, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [50]:
from torch.utils.data import DataLoader, Dataset
import torch


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [52]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_test, df_train['label'].values)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [53]:
from torch import nn


class RNNFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        rnn_out, ht = self.rnn(x) 
        # rnn_out: тензор с выходными фичами с последнего слоя для каждого t
        # h_t: тензор с последними скрытыми состояниями по слоям

        if self.use_last:
            last_tensor = rnn_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(rnn_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        
        return torch.sigmoid(out)

In [54]:
rnn_init = RNNFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(rnn_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [55]:
print(rnn_init)
print("Parameters:", sum([param.nelement() for param in rnn_init.parameters()]))

RNNFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (rnn): RNN(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 259861


In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rnn_init.to(device)
device

'cpu'

In [57]:
rnn_init = rnn_init.to(device)
rnn_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    rnn_init.train()
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = rnn_init(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    rnn_init.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Step [{i + 1}/{len(train_loader)}]. ' \
          f'Loss: {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = rnn_init(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
print('Training is finished!')

Epoch [1/5]. Step [63/63]. Loss: 0.239. Acc: 0.930. Test loss: 0.261. Test acc: 0.930
Epoch [2/5]. Step [63/63]. Loss: 0.285. Acc: 0.930. Test loss: 0.287. Test acc: 0.930
Epoch [3/5]. Step [63/63]. Loss: 0.245. Acc: 0.930. Test loss: 0.253. Test acc: 0.930
Epoch [4/5]. Step [63/63]. Loss: 0.205. Acc: 0.930. Test loss: 0.287. Test acc: 0.930
Epoch [5/5]. Step [63/63]. Loss: 0.255. Acc: 0.930. Test loss: 0.253. Test acc: 0.930
Training is finished!
